In [2]:
#CREATE AND TRAIN RRMP MODEL     DAY AHEAD Predictions Trained on REAL - TIME
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import joblib

# Load your prepared dataset
df = pd.read_csv("61761_verified.csv")

# Filter data for Sundays in July for the years 2020 and 2021
data = df[(df['Month'] == 7) & (df['Day_of_Week'] == 7) & (df['Year'].isin([2020, 2021]))]

# RRMP Model Features
features = ['Hour', 'Marginal Cost Losses', 'Marginal Cost Congestion']
X = data[features].values
y = data['LBMP'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Ridge regression model
ridge = Ridge(alpha=2.0)  # Regularization
ridge.fit(X_train, y_train)

# Save the scaler and model
joblib.dump(scaler, 'RRMP_scaler.pkl')
joblib.dump(ridge, 'RRMP_model.pkl')

['RRMP_model.pkl']

In [7]:
import joblib
import pandas as pd

# Load your saved Ridge regression model
ridge_model = joblib.load('RRMP_model.pkl')

# Load the scaler used during training
scaler = joblib.load('RRMP_scaler.pkl')

# Load the dataset with real-time values for prediction generation
daf = pd.read_csv("61761_ISO_No_LBMP.csv")
july_sundays_2022 = daf[(daf['Year'] == 2022)]

# Prepare the features
features = ['Hour', 'Marginal Cost Losses', 'Marginal Cost Congestion']
X = july_sundays_2022[features].values

# Scale the features
X_scaled = scaler.transform(X)

# Make predictions using the loaded Ridge regression model
predictions_2022 = ridge_model.predict(X_scaled)

# Convert predictions to a DataFrame with the same index
predictions_df = pd.DataFrame(predictions_2022, columns=['Predicted_LBMP'], index=july_sundays_2022.index)

# Combine predictions with the original DataFrame
results_df = july_sundays_2022.copy()
results_df['Predicted_LBMP'] = predictions_df['Predicted_LBMP']

# Save the DataFrame to a CSV file
results_df.to_csv('RRMP_DayAhead_Predictions.csv', index=False)

print("Predictions saved to 'RRMP_DayAhead_Predictions.csv'.")


Predictions saved to 'RRMP_DayAhead_Predictions.csv'.
